# Getting Started

## Running Beamlime

Run the `beamlime` command, and specify which workflow to use.
Usually this would be done from a terminal.
The current setup plots the results of the workflow and saves them to a file, which can we specify using the `--image-path` flag:

In [ ]:
import sys
import os

sys.path.append("../../tests")

from conftest import get_path

os.environ["TEST_NEXUS_FILE_PATH"] = get_path("ymir_detectors.nxs").as_posix()

In [ ]:
!beamlime \
    --workflow dummy \
    --nexus-file-path $TEST_NEXUS_FILE_PATH \
    --image-path-prefix reduction-result \
    --fill-dummy-data

There is another command ``beamlime-raw-detectors`` but it does not work with ymir file.
```bash
!beamlime-raw-detectors \
    --fake-listener \
    --static-file-path loki.hdf  \
    --nexus-file-path loki.hdf  \
    --fill-dummy-data
```

The dummy workflow returns a single result named "reduction-result.png", which is added as a suffix to the output file name:

![Plot of reduction result](reduction-result.png)

## Adding Workflow Constructors

Python packages can add workflow constructors as plugins to Beamlime.
To do this, create a Python package with a `beamlime.workflow_plugin` entry point in the `pyproject.toml` file.
This is how the "dummy" workflow is added to Beamlime:

```python
# In beamlime's pyproject.toml
[project.entry-points."beamlime.workflow_plugin"]
dummy = "beamlime.workflow_protocols:DummyLiveWorkflow"
```

Above, 'dummy' is the name of the workflow (which can then be passed to `beamlime --workflow`), and 'beamlime.workflow_protocols:' is the name of the Python package and module containing the workflow constructor.
As an example, for a Loki monitor workflow plugin provided by `esssans`, the entry point might look as follows:

```python
# In esssans' pyproject.toml
[project.entry-points."beamlime.workflow_plugin"]
ess-loki-monitor = "ess.loki.live:LokiMonitorWorkflow"
```

where `live_workflow` must adhere to the [beamlime.LiveWorkflow](https://scipp.github.io/beamlime/generated/classes/beamlime.LiveWorkflow.html) protocol:

In [ ]:
import inspect

from beamlime import LiveWorkflow

print(inspect.getsource(LiveWorkflow))

This can be fulfilled by a class with
- a constructor that accepts a nexus file path
- ``__call__`` method that accepts `JSONGroup`s of each types, (nxlog, nxevent)
- ``__call__`` method that returns a dictionary of a plottable results: [beamlime.WorkflowResult](https://scipp.github.io/beamlime/generated/classes/beamlime.WorkflowResult.html).